In [1]:
from ftplib import FTP
import gzip

from tqdm import tqdm
import time
import itertools
import pickle

import pandas as pd
import numpy as np

from rdkit import Chem
from mordred import Calculator, descriptors

In [ ]:
# Connecting to PubChem FTP Server
ftp = FTP('ftp.ncbi.nlm.nih.gov')
ftp.login()  # Anonymous login
ftp.timeout = 120
# Switch to the path of the PubChem database
ftp.cwd('/pubchem/Compound/CURRENT-Full/SDF')

In [ ]:
filename = 'Compound_004000001_004500000.sdf.gz' 
local_path = '/home/is/pei-ga/Sony/AE/'+filename

with open(local_path, 'wb') as file:
    ftp.retrbinary('RETR ' + filename, file.write)

In [ ]:
ftp.quit()

In [2]:
def mord(mol):
    desc = calc(mol)
    mordred = np.asarray(desc).astype(float)
    mordred = np.round(mordred, 2)
    return mordred

In [ ]:
sv_dir = 'Compound_000000001_000500000'
calc = Calculator(descriptors, ignore_3D= True)
# start_index = 0
# end_index = 50000
with gzip.open(sv_dir+'.sdf.gz', 'rb') as f: 
    suppl = Chem.ForwardSDMolSupplier(f)
# Use itertools.islice to intercept a specified range of data
#    subset = list(itertools.islice(suppl, start_index, end_index))
    array_list = list(tqdm((mord(mol) for mol in suppl if mol is not None)))

In [5]:
sv_path = '/home/is/pei-ga/Sony/AE/split_0.pkl'
with open(sv_path, "wb") as handle:
    pickle.dump(array_list, handle, protocol=pickle.HIGHEST_PROTOCOL)